In [1]:
import sys
sys.path.append('../')

In [ ]:
from utils.utils import Utils
from utils.preprocess import Preprocess

In [16]:
%%time
utils = Utils()
pre = Preprocess(mode="clean")

CPU times: user 2.33 s, sys: 86.7 ms, total: 2.41 s
Wall time: 2.42 s


In [26]:
%%time
data = utils.read_data("../dataset/track-1-essay-empathy-train-normalized.csv")
data

CPU times: user 30.2 ms, sys: 71 µs, total: 30.3 ms
Wall time: 30.4 ms


,message_id,response_id,empathy,distress,empathy_bin,distress_bin,emotion_label,essay,gender,education,...,income,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
0,R_1hGrPtWM4SumG0U_1,R_1hGrPtWM4SumG0U,5.667,4.375,1,1,sadness,it is really disheartening to read about thes...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
1,R_1hGrPtWM4SumG0U_2,R_1hGrPtWM4SumG0U,4.833,4.875,1,1,joy,the phone lines from the suicide prevention l...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
2,R_1hGrPtWM4SumG0U_3,R_1hGrPtWM4SumG0U,5.333,3.500,1,0,neutral,"no matter what your heritage , you should be ...",1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
3,R_1hGrPtWM4SumG0U_4,R_1hGrPtWM4SumG0U,4.167,5.250,1,1,fear,it is frightening to learn about all these sh...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
4,R_1hGrPtWM4SumG0U_5,R_1hGrPtWM4SumG0U,5.333,4.625,1,1,neutral,the eldest generation of russians are not bei...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,R_DHy2Rcz9Hym8jgl_1,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,joy,"These days , women are winning in sports . Th...",2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1856,R_DHy2Rcz9Hym8jgl_2,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,anger,I hate SIS . They are a group full of hate an...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1857,R_DHy2Rcz9Hym8jgl_3,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,disgust,This is just disgusting . I cannot believe th...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1858,R_DHy2Rcz9Hym8jgl_4,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,surprise,I feel like the world is so corrupt that it n...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429


In [28]:
post_no = 678
data.essay.values.tolist()[post_no]

' I am not a fan of Muslim migrants myself . I find most of Arab culture backwards and antithetical to American values . This is pretty fucked up , though . This article feels like something out of the 1960s . Takes you consider the cyclical and incremental nature of human history . Despite our technological advances we fall prey to the same prejudices and biases again and again . Before we were burning Native Americans , then the Irish , then Blacks , and now Muslims . Human progress at work .'

In [19]:
%%time
normalized_data = pre.normalize_corpus(data.copy(), column_name="essay")
normalized_data

CPU times: user 43.4 s, sys: 14.5 ms, total: 43.4 s
Wall time: 43.4 s


,message_id,response_id,empathy,distress,empathy_bin,distress_bin,emotion_label,essay,gender,education,...,income,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
0,R_1hGrPtWM4SumG0U_1,R_1hGrPtWM4SumG0U,5.667,4.375,1,1,sadness,it is really disheartening to read about thes...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
1,R_1hGrPtWM4SumG0U_2,R_1hGrPtWM4SumG0U,4.833,4.875,1,1,joy,the phone lines from the suicide prevention l...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
2,R_1hGrPtWM4SumG0U_3,R_1hGrPtWM4SumG0U,5.333,3.500,1,0,neutral,"no matter what your heritage , you should be ...",1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
3,R_1hGrPtWM4SumG0U_4,R_1hGrPtWM4SumG0U,4.167,5.250,1,1,fear,it is frightening to learn about all these sh...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
4,R_1hGrPtWM4SumG0U_5,R_1hGrPtWM4SumG0U,5.333,4.625,1,1,neutral,the eldest generation of russians are not bei...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,R_DHy2Rcz9Hym8jgl_1,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,joy,"These days , women are winning in sports . Th...",2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1856,R_DHy2Rcz9Hym8jgl_2,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,anger,I hate SIS . They are a group full of hate an...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1857,R_DHy2Rcz9Hym8jgl_3,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,disgust,This is just disgusting . I cannot believe th...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1858,R_DHy2Rcz9Hym8jgl_4,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,surprise,I feel like the world is so corrupt that it n...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429


In [29]:
normalized_data.essay.values.tolist()[post_no]

' I am not a fan of Muslim migrants myself . I find most of Arab culture backwards and antithetical to American values . This is pretty fucked up , though . This article feels like something out of the 1960s . Takes you consider the cyclical and incremental nature of human history . Despite our technological advances we fall prey to the same prejudices and biases again and again . Before we were burning Native Americans , then the Irish , then Blacks , and now Muslims . Human progress at work .'

In [25]:
normalized_data.to_csv("../dataset/track-1-essay-empathy-train-normalized.csv", index=False)